# Load books

In [1]:
books = []

for i in range(1, 8):
    with open(f'books/{i}.txt', 'r') as fs:
        books.append(fs.read())
    print(f'{i}.txt', len(books[-1]))

print()
print(books[0][0:1000])

1.txt 439478
2.txt 492297
3.txt 625735
4.txt 1100515
5.txt 1499410
6.txt 987992
7.txt 1140011

M r. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense.

Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere.

The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found o

In [12]:
# Some long paragraphs will be truncated during training. 
# To avoid lossing data, we split them into shorter ones.

import nltk
from transformers import GPT2Tokenizer

# Download the necessary NLTK data
nltk.download('punkt')

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def split_text_at_sentences(text, max_tokens=700):
    # Tokenize the text into sentences
    sentences = nltk.tokenize.sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_tokens = []
    
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        
        # Check if adding this sentence would exceed the max token limit
        if len(current_tokens) + len(sentence_tokens) > max_tokens:
            # If so, finalize the current chunk and start a new one
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_tokens = []
        
        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_tokens.extend(sentence_tokens)
    
    # Add the last chunk if there's any content left
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
chucked_books = []
for i, book in enumerate(books):
    chunked_book = ''
    chunks = split_text_at_sentences(book.replace("\n", " "))
    for chunk in chunks:
        chunked_book += chunk + "\n"
    chucked_books.append(chunked_book)

# Merge to train and eval

In [16]:
num_train_books = 4
train = ''
eval =''
for i in range(num_train_books):
    train += chucked_books[i]
for j in range(len(chucked_books)-num_train_books):    
    eval += chucked_books[num_train_books+j]

with open(f'data_files/train.txt', 'w') as fs:
    fs.write(train)

with open(f'data_files/eval.txt', 'w') as fs:
    fs.write(eval)

with open(f'data_files/full.txt', 'w') as fs:
    fs.write(train + eval)

In [17]:
def count_lines_in_file(filename):
    with open(filename, 'r') as file:
        line_count = sum(1 for line in file)
    return line_count
 
# Example usage:
filename = 'your_text_file.txt'
train_number_of_lines = count_lines_in_file('data_files/train.txt')
full_number_of_lines = count_lines_in_file('data_files/full.txt')
eval_number_of_lines = count_lines_in_file('data_files/eval.txt')
print(f'The train file has {train_number_of_lines} lines.')
print(f'The full file has {full_number_of_lines} lines.')
print(f'The eval file has {eval_number_of_lines} lines.')

The train file has 1033 lines.
The full file has 2370 lines.
The eval file has 1337 lines.
